In [43]:
import numpy as np
import math

In [48]:
M = np.array([[5,2,4,4,3],[3,1,2,4,1],[2,np.nan,3,1,4],[2,5,4,3,5],[4,4,5,4,np.nan]]) #let the blank elements be NaN
M_original = np.copy(M)
print(M)

#Preprocessing:

#first, loop through rows
for i in range(len(M[0,:])):
    row_mean = np.nanmean(M[i,:]) #compute the mean for all non-NaN elements 
    print('mean of row',i,row_mean)
    
    for j in range(len(M[0])):
        if not math.isnan(M[i,j]): #check if an element is NaN
            M[i,j] -= row_mean
    
print(M)
#next, the columns
for j in range(len(M[0])):
    column_mean = np.nanmean(M[:,j])
    print('mean of column',j,column_mean)
    
    for i in range(len(M[0,:])):
        if not math.isnan(M[j,i]):
            M[j,i] -= column_mean
    
print(M)

print(np.nanmean(M_original))
print(np.nanmean(M))

[[ 5.  2.  4.  4.  3.]
 [ 3.  1.  2.  4.  1.]
 [ 2. nan  3.  1.  4.]
 [ 2.  5.  4.  3.  5.]
 [ 4.  4.  5.  4. nan]]
mean of row 0 3.6
mean of row 1 2.2
mean of row 2 2.5
mean of row 3 3.8
mean of row 4 4.25
[[ 1.4  -1.6   0.4   0.4  -0.6 ]
 [ 0.8  -1.2  -0.2   1.8  -1.2 ]
 [-0.5    nan  0.5  -1.5   1.5 ]
 [-1.8   1.2   0.2  -0.8   1.2 ]
 [-0.25 -0.25  0.75 -0.25   nan]]
mean of column 0 -0.07000000000000002
mean of column 1 -0.44500000000000006
mean of column 2 0.433
mean of column 3 -0.0536
mean of column 4 0.2589
[[ 1.47   -1.53    0.47    0.47   -0.53  ]
 [ 1.245  -0.755   0.245   2.245  -0.755 ]
 [-0.933      nan  0.067  -1.933   1.067 ]
 [-1.7464  1.2536  0.2536 -0.7464  1.2536]
 [-0.5089 -0.5089  0.4911 -0.5089     nan]]
3.260869565217391
0.003278260869565223


In [53]:
#Initialization

init_value = np.sqrt(np.nanmean(M)/2) #assuming d = 2, try many d values later 
print(init_value)
#dont use the init value for now, try later 

U = np.ones((5,2))
V = np.ones((2,5))
print(U)
print(V)

0.0404861758478448
[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [70]:
#Optimization
print(M)
print(U)
print(V)

RMSE = 0
P = np.zeros((5,5))
diff = np.zeros((5,5))
for r in range(5):
    for j in range(5):
        print(M_original[r,j])
        for k in range(2):
            if k != j:
            #print('performing operation with P')
                P[r,j] = P[r,j] + U[r,k]*V[k,j] 
            else:
                P[r,j] = P[r,j] +  U[r,j]*V[k,j] 
        if not math.isnan(M_original[r,j]):        
            RMSE = RMSE + (M_original[r,j] - P[r,j])**2
        #P[r,j] = P[r,j] + 1
        #print(M[r,j])
        
print(RMSE)
print(P)

[[ 1.47   -1.53    0.47    0.47   -0.53  ]
 [ 1.245  -0.755   0.245   2.245  -0.755 ]
 [-0.933      nan  0.067  -1.933   1.067 ]
 [-1.7464  1.2536  0.2536 -0.7464  1.2536]
 [-0.5089 -0.5089  0.4911 -0.5089     nan]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
5.0
2.0
4.0
4.0
3.0
3.0
1.0
2.0
4.0
1.0
2.0
nan
3.0
1.0
4.0
2.0
5.0
4.0
3.0
5.0
4.0
4.0
5.0
4.0
nan
75.0
[[2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2.]]
